In [1]:
%pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [2]:
import stop_words as sw

[nltk_data] Downloading package stopwords to /Users/tavin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/tavin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
sw.stop_words("The real words are the friends we made along the way.")

['real', 'words', 'friends', 'made', 'along', 'way']